# Import library
# Set params 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import skfuzzy as fuzz
import random
import os
import cv2 as cv2
import numpy as np
import math as math
import time
from sklearn.metrics.pairwise import euclidean_distances
import operator

C = 5
D = 3
M = 2.0
error = 0.01
maxiter = 10

# Read image 

In [ ]:
# img = cv2.imread('burma.jpg')
# rgb_img = img.reshape((img.shape[0] * img.shape[1]), 3)

# print(rgb_img)
# print(img.shape[0], img.shape[1], img.shape[2])

# N = img.shape[0] * img.shape[1]

# cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
#     rgb_img.T, 5, 2.0, error = 0.01, maxiter = 1000, init=None, seed=42)



In [2]:
img = cv2.imread('brain3.jpg')
N = img.shape[0] * img.shape[1]
rgb_img = img.reshape(N, D)


In [3]:
X = rgb_img


# Init V with C clusters's centers, D dimension each cluster center 

def initV():
    
    V = np.random.rand(C,D)
    
    return V

def initU(X, V, N, C):
    
    U = np.ndarray(shape=(N, C), dtype=float)
    
    p = 2.0/(M-1.0)
    
    for i in range(N):
        x = list(X[i])
        distances = [np.linalg.norm(list(map(operator.sub, x, V[j]))) for j in range(C)]
        for j in range(C):
            denom = sum([math.pow((distances[j]/distances[k]), p) for k in range(C)])
            U[i][j] = 1.0/denom
    
    return U 

def normalizeU(U):
    
    for i in range(N):
        sigma = sum(U[i])
        U[i] = U[i] / sigma
        
    return U

def normalizeX(X):
    
    minX = np.amin(X)
    maxX = np.amax(X)
    
    X = (X - minX) / (maxX - minX)
    
    return X
    
def updateU(X, V, U):
    
    p = float(2.0/(M-1.0))
    
    for i in range(N):
        x = list(X[i])
        distances = [np.linalg.norm(list(map(operator.sub, x, V[j]))) for j in range(C)]
        for j in range(C):
            denom = sum([math.pow((distances[j]/distances[k]), p) for k in range(C)])
            U[i][j] = float(1.0/denom)
            
    return U
    
    
def updateV(X, U):
    
    U_col = zip(*U)
    V = list()
    
    for u in U_col:
        u_exp = [u1 ** M for u1 in u]
        denom = sum(u_exp)
        temp = list()
        for i in range(N):
            X[i] = list(X[i])
            prod = [u_exp[i] * x for x in X[i]]
            temp.append(prod)
        numer = map(sum, zip(*temp))
        Vj = [a/denom for a in numer]
        V. append(Vj)
    
    return V

def fuzzy_cmeans(X, C, M, error, maxiter):
    
    V = initV()
    U = initU(X,V,N,C)
    U = normalizeU(U)
    X = normalizeX(X)
        
    iter = 0
    
    print('starting loop')
    
    while iter <= maxiter:
        
        V = updateV(X, U)
        print('V[%r]:' % (iter))
        print(V)
        U_old = U
        U = updateU(X, V, U)
        print('U[%r]:' % (iter))
        print(U)
        iter += 1
        
        print('end loop')

    return V, U
        
V, U = fuzzy_cmeans(X, C, M, error, maxiter)
        

starting loop
V[0]:
[[0.3216593605082588, 0.3216593605082588, 0.3216593605082588], [0.31961088602289434, 0.31961088602289434, 0.31961088602289434], [0.3201117605410365, 0.3201117605410365, 0.3201117605410365], [0.3200888535024665, 0.3200888535024665, 0.3200888535024665], [0.319047950279811, 0.319047950279811, 0.319047950279811]]
U[0]:
[[0.20115009 0.19963564 0.20000435 0.19998747 0.19922245]
 [0.20115009 0.19963564 0.20000435 0.19998747 0.19922245]
 [0.20115009 0.19963564 0.20000435 0.19998747 0.19922245]
 ...
 [0.19756027 0.20077312 0.19998034 0.2000165  0.20166977]
 [0.19756027 0.20077312 0.19998034 0.2000165  0.20166977]
 [0.19806584 0.20061289 0.19998559 0.20001422 0.20132145]]
end loop
V[1]:
[[0.3248941278220418, 0.3248941278220418, 0.3248941278220418], [0.3182459464808307, 0.3182459464808307, 0.3182459464808307], [0.3202372776112315, 0.3202372776112315, 0.3202372776112315], [0.32014562401620517, 0.32014562401620517, 0.32014562401620517], [0.3161566270855858, 0.3161566270855858, 0

In [ ]:
V = np.asarray(V)

minX = np.amin(rgb_img)
maxX = np.amax(rgb_img)

V = V * (maxX - minX)

print(V)

u = U.T
cntr = V

print(cntr.shape[0], cntr.shape[1])
print(u.shape[0], u.shape[1])

In [ ]:
def change_color_fuzzycmeans(u, cntr):
    img = []
    for pix in u.T:
        img.append(cntr[np.argmax(pix)])
    return img

def drawContours(seg_img_1d, areaPixels):
    
    seg_img_1d_copy = seg_img_1d.copy()
    image, contours, hierachy = cv2.findContours(seg_img_1d_copy, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    for idx in np.arange(len(contours)):
        area = cv2.contourArea(contours[idx])
        if(area >= 0 and area <= areaPixels):
            cv2.drawContours(seg_img_1d_copy, contours, idx, (0,0,0), -1)
            
    return seg_img_1d_copy

def clearBorder(img1):
    
    r = 2
    img1_copy = img1.copy()
    image, contours, hierarchy = cv2.findContours(img1_copy, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    imgRows = img1_copy.shape[0]
    imgCols = img1_copy.shape[1]
    
    contourList = []
    
    for idx in np.arange(len(contours)): 
        
        cnt = contours[idx]
        
        for pt in cnt: 
            rowCnt = pt[0][1]
            colCnt = pt[0][0]
            
            check1 = (rowCnt >= 0 and rowCnt < r) or (rowCnt >= imgRows-1-r and rowCnt < imgRows)
            check2 = (colCnt >= 0 and colCnt < r) or (colCnt >= imgCols-1-r and colCnt < imgCols)
            
            if check1 or check2: 
                contourList.append(idx)
                break
        
    for idx in contourList:    
        cv2.drawContours(img1_copy, contours, idx, (0,0,0), -1)
    
    return img1_copy
    

def imfill(img2):
    
    im_floodfill  = img2.copy()
    
    h, w = im_floodfill.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)
    
    cv2.floodFill(im_floodfill, mask, (0,0), 255)
    
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)
    
    im_out = img2 | im_floodfill_inv
    
    return im_out
    
    

In [ ]:
new_img = change_color_fuzzycmeans(u, cntr)
fuzzy_img = np.reshape(new_img, (231, 218, 3)).astype(np.uint8)

ret, seg_img = cv2.threshold(fuzzy_img, np.max(fuzzy_img) -1 , 255, cv2.THRESH_BINARY)

seg_img_1d = seg_img[:,:,1]

img1 = drawContours(seg_img_1d, 100)
img2 = clearBorder(img1)
img3 = imfill(img2)

plt.imshow(img3)



In [ ]:
# temp = [[1,2,3,4], [5,6,7,8], [9,10,11,12]]

# temp1 = map(sum, zip(*temp))

# print(temp)

# for i in temp1:
#     print(i)

# a = [1,2,3,4]
# b = [2,3,4,5]

# c = list(map(operator.sub, a, b))

# d = np.linalg.norm(c)
# print(d)
    
# a = [[1,2,3,4], [5,6,7,8]]

# b = zip(*a)

# for k in b:
#     x = list(k)
#     y = [z ** 2 for z in x]
#     print(y)

# U = np.ndarray(shape=(3,2),dtype=float)

# U[0] = [1,2]
# U[1] = [3,4]
# U[2] = [5,6]

# print(U)

# T = zip(*U)

a = 1

print ('Result is : %r ' % (a))


# Auxillary functions 